In [22]:
import requests
import json
import urllib.parse
import datetime
import sqlite3
import os
import re
import re 
import xml.etree.ElementTree as ET
from collections import Counter
from Bio import Entrez
import mercury as mr
from bs4 import BeautifulSoup
import concurrent.futures
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import to_hex

In [23]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    os.makedirs(path)

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454_jp_v8.db")
cur = con.cursor()

In [24]:
## Tabellen Schema erstellen
#------------------- Tabelle Zitationen --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Zitationen(PMID INTEGER, Jahr YEAR, Anzahl INTEGER, UNIQUE(PMID,Jahr) ON CONFLICT REPLACE)")

#------------------- Tabelle MeSH --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS MeSH(UI VARCHAR, Term TEXT, Jahr_MeSH YEAR, EK INTEGER, NEK INTEGER, UNIQUE(UI) ON CONFLICT REPLACE)")

#------------------- Tabelle Mesh - Dokument --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS MeSH_Dok(UI VARCHAR, PMID INTEGER, EK_Dok INTEGER, NEK_Dok INTEGER, UNIQUE(UI,PMID) ON CONFLICT REPLACE)")
con.commit()

In [25]:
def getAllUIsFromDB():
    cur.execute("SELECT UI FROM MeSH")
    MeSHs = [x[0] for x in cur.fetchall()]
    return MeSHs

# MeSH-Dokumenten Zuordnung
def DB_Insert_MeSH_PMID(UI, PMID):
    try:
        cur.execute("INSERT INTO MeSH_Dok(UI, PMID) VALUES ('"+str(UI)+"', '"+str(PMID)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return  

# Zitationsdaten für ein Dokument
def DB_Insert_Zitationen(PMID, Zitationsdaten):
    try:
        cur.executemany("INSERT INTO Zitationen(PMID, Jahr, Anzahl) VALUES ('"+str(PMID)+"', ?, ?)", Zitationsdaten)
        con.commit()
        # Insert erfolgreich
    except:
        # Zitationsdaten schon in DB
        return  
    
# Analyseergebnisse in DB einfügen
def DB_Insert_Analyse_Per_MeSH(ui, analyseDaten, EKK, NEKK):
    try:
        cur.execute("UPDATE MESH SET EK = '"+str(EKK) +"', NEK = '"+str(NEKK)+"' WHERE UI = '"+str(ui)+"'")
        con.commit()
        cur.executemany("UPDATE MeSH_Dok SET EK_Dok = ? , NEK_Dok = ? WHERE PMID = ?", analyseDaten)
        con.commit()
        # Insert erfolgreich
    except:
        # Zitationsdaten schon in DB
        return  
    
def DB_Get_Zitationen(PMID):
    cur.execute("SELECT Jahr, Anzahl FROM Zitationen WHERE PMID = '" + str(PMID) +"'")
    citations = cur.fetchall()
    return citations
    
def make_request(url):
    anfrage = requests.get(url, headers={"Accept": "application/json"})
    antwort_Daten = anfrage.json()
    return antwort_Daten

In [26]:
global ui_db 
ui_db = getAllUIsFromDB()

In [27]:
# Funktion ermittelt alle UI: {Term, Link, dateEstablished}, die zu Suchwort passen
def suchwort_MeSH_Auswahl(suchwort):
    url = "https://id.nlm.nih.gov/mesh/lookup/descriptor?label="+str(suchwort)+"&match=contains&year=current&limit=50"
    anfrage = requests.get(url, headers={"Accept": "application/json"})
    antwort_Daten_1 = anfrage.json()
    meshs = {}
    urls = []
    for antwort in antwort_Daten_1:
        link = antwort['resource']
        ui = str(re.findall("^.*mesh\/(.*)$", link)[0])
        meshs[ui] = {"Term": antwort['label'], "Link": link}
        urls.append(meshs[ui]["Link"]+str(".json"))
        if ui not in ui_db:
            with concurrent.futures.ThreadPoolExecutor() as executor:  
                futures = [executor.submit(make_request, url) for url in urls]
            responses = [future.result() for future in concurrent.futures.as_completed(futures)]
            for url, antwort_Daten in zip(urls, responses):
                ui = str(re.findall("^.*mesh\/(.*).json$", url)[0])
                dateEstablished = datetime.datetime.strptime(antwort_Daten["dateCreated"], "%Y-%m-%d").year
                meshs[ui]["dateEstablished"] = dateEstablished
                cur.execute("INSERT INTO MeSH(UI, Term, Jahr_MeSH) VALUES ('"+str(ui)+"', '"+str(meshs[ui]["Term"])+"', '"+str(dateEstablished)+"')")
                con.commit()
    return meshs


In [28]:
# Eine Funktion, um den Inhalt der Tabelle der Zitationen in eine Liste umzuwandeln
def getCitationForPMID(PMID):
    try:
        # HTTP-Anfrage an die URL senden
        url = "https://pubmed.ncbi.nlm.nih.gov/?linkname=pubmed_pubmed_citedin&from_uid=" + str(PMID)
        response = requests.get(url)

        # Den HTML-Inhalt der Seite parsen
        soup = BeautifulSoup(response.text, "html.parser")

        # Die Tabelle mit der angegebenen ID auswählen
        table = soup.find("table", {"id": "timeline-table"})

        # Eine Liste, um die Daten aus der Tabelle zu speichern
        data_list = []

        # Durch die Zeilen der Tabelle iterieren
        for row in table.find_all("tr"):
            row_data = []
            # Durch die Zellen (Spalten) der Zeile iterieren
            for cell in row.find_all("td"):
                row_data.append(cell.text)
            if row_data != []:
                data_list.append(row_data)
        DB_Insert_Zitationen(PMID, data_list)
    except:
        pass

In [55]:
# Funktion, die die Dokumente ermittelt, die mit dem MeSH verknüpft sind und vor der Aufnahme des MeSHs veröffentlicht wurden (Von 1900 an)
def Dokumente_mit_MeSH(meshs, count_max):
    for mesh in meshs:
        if mesh not in ui_db:
            print("Zu dem MeSH mit der UI " + str(mesh) + " werden folgend "+ str(count_max) + " Dokumente ermittelt, welchem dieses MeSH zugeordnet ist.")
            Jahr_MeSH = meshs[mesh]["dateEstablished"]
            Term = meshs[mesh]["Term"]
            # API ist auf die ersten 10000 Dokumente limitiert; esearch ist eine Möglichkeit an alle Dokumente zu kommen (wenn es denn mehr gibt) allerdings muss man auf einer UNIX Umgebung arbeiten https://dataguide.nlm.nih.gov/edirect/edirect-vs-e-utilities.html
            # Abfrage von Dokumente, die vor der Aufnahme des MeSHs veröffentlicht wurden (Von 1900 an)
            url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=("+urllib.parse.quote(Term)+"[MeSH+Terms])+AND+((\"1900\"[Date+-+Publication]+%3A+\""+str(Jahr_MeSH)+"\"[Date+-+Publication]))&retmax="+str(count_max)+"&sort=relevance&retmode=json&api_key=177864321075a405e5c56e95464bdf959f08"
            response = requests.get(url)
            daten = json.loads(response.text)
            pmids = ""
            for PMID in daten['esearchresult']['idlist']:
                if pmids == "":
                    pmids += str(PMID)
                else:
                    pmids += ", " + str(PMID)
                DB_Insert_MeSH_PMID(mesh, PMID)
                getCitationForPMID(PMID)
            print("Für das MeSH " + str(mesh) + " wurden die PMIDs " + str(pmids) + " und deren Zitationshäufigkeiten ermittelt.")



In [49]:

def Datenerhebung(suchwort):
    meshs = suchwort_MeSH_Auswahl(suchwort)
    # mr.Markdown(f"### Es wurden {len(meshs)} MeSH in Verbindung zu Ihrem Suchwort gefunden.")
    if len(meshs) != 0:
        mr.Markdown(f"### Es folgt eine Auflistung der MeSH, die im Zusammenhang zu dem Suchbegriff stehen:")
        for mesh in meshs:
            print(mesh, meshs[mesh]['Term'])
    return meshs
        

In [31]:
# Datenerhebung("head")

In [32]:
def Zitationen_emergent(PMID, Jahr_MeSH):
    url = "SELECT sum(Anzahl) FROM Zitationen WHERE PMID = "+str(PMID)+" and Jahr <= '"+str(Jahr_MeSH)+ "'"
    cur.execute(url)
    antwort = cur.fetchall()[0][0] 
    Anzahl = antwort if antwort != None else 0 
    return Anzahl

def Zitationen_nicht_emergent(PMID, Jahr_MeSH):
    url = "SELECT sum(Anzahl) FROM Zitationen WHERE PMID = "+str(PMID)+" and Jahr > '"+str(Jahr_MeSH)+ "'"
    cur.execute(url)
    antwort = cur.fetchall()[0][0] 
    Anzahl = antwort if antwort != None else 0 
    return Anzahl

def Aufbereitung():
    cur.execute("SELECT UI, Jahr_MeSH FROM MeSH WHERE EK IS NULL ")
    uis = cur.fetchall()
    # uis = [x[0] for x in cur.fetchall()]
    for ui, jahr in uis:
        cur.execute("SELECT PMID FROM MeSH_Dok WHERE UI = '" + str(ui)+ "'")
        PMIDs = [x[0] for x in cur.fetchall()]
        EKK = 0
        NEKK = 0
        analyseDaten = []
        for PMID in PMIDs:
            EK_Dok = Zitationen_emergent(PMID, jahr)
            NEK_Dok = Zitationen_nicht_emergent(PMID, jahr)
            analyseDaten.append([EK_Dok, NEK_Dok, PMID])
            EKK += EK_Dok
            NEKK += NEK_Dok
        DB_Insert_Analyse_Per_MeSH(ui, analyseDaten, EKK, NEKK)

In [33]:
def Analyse(meshs):  
    mesh_lst = ""  
    for mesh in meshs:
        mesh_lst += ",'" + mesh + "'"
    mesh_lst = mesh_lst[1:]
    cur.execute("SELECT EK_Dok, EK_Dok+NEK_Dok, PMID, UI FROM MeSH_Dok WHERE UI IN ("+str(mesh_lst)+") ORDER BY EK_Dok+NEK_Dok DESC")
    Daten = cur.fetchall()
    EK = [x[0] for x in Daten]
    Gesamt = [x[1] for x in Daten]
    Daten_gefiltert = [x for x in Daten if x[1] != 0] # ohne Nullen in gesamt
    EK_gefiltert = [x[0] for x in Daten_gefiltert]
    Gesamt_gefiltert = [x[1] for x in Daten_gefiltert]
    # print(Daten)
    kor = np.corrcoef(EK, Gesamt)[0,1]
    kor_gefiltert = np.corrcoef(EK_gefiltert, Gesamt_gefiltert)[0,1]
    mr.Markdown(f"### Folgend werden die Dokumente nach der Gesamtzitationsanzahl absteigend sortiert. Dazu werden auch die emergenten Zitationszahlen ausgegeben.") 
    cmap = plt.get_cmap('summer')
    normalized_values_gesamt = [(value - min(Gesamt)) / (max(Gesamt) - min(Gesamt)) for value in Gesamt]
    normalized_values_emergent = [(value - min(EK)) / (max(EK) - min(EK)) for value in EK]
    
    # normalized_values_gesamt = [(value - min(Gesamt_gefiltert)) / (max(Gesamt_gefiltert) - min(Gesamt_gefiltert)) for value in Gesamt_gefiltert]
    # normalized_values_emergent = [(value - min(EK_gefiltert)) / (max(EK_gefiltert) - min(EK_gefiltert)) for value in EK_gefiltert]
    colors_gesamt = [cmap(value) for value in normalized_values_gesamt]
    colors_emergent = [cmap(value) for value in normalized_values_emergent]
    body = ""
    i = 0
    for dat in Daten_gefiltert:
        body += "<tr><td>"+str(dat[3])+"</td> <td>"+str(dat[2])+"</td> <td bgcolor="+str(to_hex(colors_gesamt[i]))+">"+str(dat[1])+"</td> <td bgcolor="+str(to_hex(colors_emergent[i]))+">"+str(dat[0])+"</td></tr>"
        i += 1
    
    table = """<table>
    <thead>
        <tr>
        <th>MeSH UI</th>
        <th>PMID</th>
        <th>Anzahl Zitationen (gesamt)</th>
        <th>Anzahl emergenter Zitationen</th>
        </tr>
    </thead>
    <tbody>""" + body + """</tbody>
    </table>"""
    mr.Markdown(f"{table}")
    mr.Markdown(f"### Außerdem wurden {len(Daten)-len(Daten_gefiltert)} Dokumente 0 mal zitiert. ") 
    mr.Markdown(f"### Der Korrelationskoeffizient zwischen der Gesamtanzahl der Zitationen und der emergenten Zitationen ist: {kor}") 
    mr.Markdown(f"### Der Korrelationskoeffizient zwischen der Gesamtanzahl der Zitationen und der emergenten Zitationen ist für die Dokumente, welche mindestens einmal zitiert wurden: {kor_gefiltert}")   

In [34]:
# show_code = mr.Checkbox(label="Quelltext ein-/ausblenden", value=False)
# app = mr.App(title="Datenerhebung", description="Erheben Sie Zitationsdaten", show_code=show_code.value)

# suchwort = mr.Text(label="Suchen Sie nach Dokumenten zu einem Suchbegriff", value="Suchbegriff")
# mr.Markdown(f"## Geben Sie in nebenstehendes Textfeld einen Begriff ein, zu dem Sie Dokumente finden möchten.")
# mr.Markdown(f"## Mit Bestätigung durch 'Enter' werden die Dokumente ermittelt.")
# mr.Markdown(f"### Suchbegriff: **{suchwort.value}**")
# meshs = Datenerhebung(suchwort.value)
# # if meshs == {}:
# #     print("Es wurden keine Dokumente zu Ihrem Suchbegriff gefunden.")
# # else:
# #     Dokumente_mit_MeSH(meshs, 3)
# #     Aufbereitung()
# #     Analyse(meshs)


# Pipeline

## Geben Sie einen Suchbegriff ein

In [48]:
suchbegriff = "head"

In [50]:
meshs = Datenerhebung(suchbegriff)

### Es folgt eine Auflistung der MeSH, die im Zusammenhang zu dem Suchbegriff stehen:

D003027 Cluster Headache
D020207 Coma, Post-Head Injury
D000094222 Dropped Head Syndrome
D005270 Femur Head
D005271 Femur Head Necrosis
D006257 Head
D064087 Head Impulse Test
D016489 Head Injuries, Closed
D020197 Head Injuries, Penetrating
D060226 Head Kidney
D019416 Head Movements
D006260 Head Protective Devices
D006258 Head and Neck Neoplasms
D018475 Head-Down Tilt
D006261 Headache
D020773 Headache Disorders
D051270 Headache Disorders, Primary
D051271 Headache Disorders, Secondary
D058430 Humeral Head
D046650 Medical Subject Headings
D051299 Post-Dural Puncture Headache
D051298 Post-Traumatic Headache
D000092467 Radial Head and Neck Fractures
D013077 Sperm Head
D000077195 Squamous Cell Carcinoma of Head and Neck
D013358 Subject Headings
D018781 Tension-Type Headache
D014653 Vascular Headaches


In [54]:
Dokumente_mit_MeSH(meshs, 3)

Zu dem MeSH mit der UI D003027 werden folgend 3 Dokumente ermittelt, welchem dieses MeSH zugeordnet ist.
Für das MeSH D003027 wurden die PMIDs 1865193916235049474711 und deren Zitationshäufigkeiten ermittelt.
Zu dem MeSH mit der UI D020207 werden folgend 3 Dokumente ermittelt, welchem dieses MeSH zugeordnet ist.
Für das MeSH D020207 wurden die PMIDs 158754411147532412076410 und deren Zitationshäufigkeiten ermittelt.
Zu dem MeSH mit der UI D000094222 werden folgend 3 Dokumente ermittelt, welchem dieses MeSH zugeordnet ist.
Für das MeSH D000094222 wurden die PMIDs  und deren Zitationshäufigkeiten ermittelt.
Zu dem MeSH mit der UI D005270 werden folgend 3 Dokumente ermittelt, welchem dieses MeSH zugeordnet ist.
Für das MeSH D005270 wurden die PMIDs 962648236676479463860 und deren Zitationshäufigkeiten ermittelt.
Zu dem MeSH mit der UI D005271 werden folgend 3 Dokumente ermittelt, welchem dieses MeSH zugeordnet ist.
Für das MeSH D005271 wurden die PMIDs 702377563867092045406 und deren Zita